In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import csv
import re
import pandas as pd
import numpy as np

from selenium.webdriver.chrome.options import Options


driver = webdriver.Chrome()
# 웹드라이버 설정 (크롬 드라이버 경로 지정)
#driver = webdriver.Chrome()

id = '2021204008'
pw = 'dbssk9182!'
# 연도 선택
year = '2024'
semester = '3'  #(f12로 여름학기의 value값을 확인후 사용)



# 로그인 페이지 열기
login_url = 'https://klas.kw.ac.kr/usr/cmn/login/LoginForm.do?redirectUrl=https://www.kw.ac.kr/include/loginCallback.jsp'
driver.get(login_url)

# 아이디와 비밀번호 입력
id_input = driver.find_element(By.ID, 'loginId')
id_input.send_keys(id)

pw_input = driver.find_element(By.ID, 'loginPwd')  
pw_input.send_keys(pw)

# 로그인 버튼 클릭
login_button = driver.find_element(By.CLASS_NAME, 'btn')
login_button.click()

time.sleep(2)

In [55]:
# List of labels
labels_list = [
    "교과목명",   #!
    "년도학기",   #!
    "이수구분",
    "학정번호",   #!
    "담당교수",
    "강의소개",  
    "학점/시간",  #!
    "강의시간 및 강의실",  #!
    "연락처",
    "이동전화",
    "이메일",
    "담당조교",
    "장애학생 지원",
    "교과목 개요",
    "대표역량",
    "학습목표 및 학습방법",
    "학습성과",
    "영어강의 비율",
    #평가방법 비율 (숫자)
    "출석",
    "중간고사",
    "기말고사",
    "과제보고서",
    "수업태도",
    "Quiz",
    "기타1",
    #boolean
    "영어강의",
    "중국어 강의",
    "일본어 강의",
    "TBL강의",
    "PBL강의",
    "세미나강의",
    "소규모강의",
    "융합강의",
    "팀티칭강의",
    "일학습병행강의",
    "외국인전용강의",
    "서울권역e-러닝",
    "실험실습실기설계강의",
    "집중이수제강의",
    "원격수업 (100%)",  #!
    "원격수업 (50% 이상)",  #!
    "100%대면강의",  #!
    "대면+실시간 화상강의",  #!
    "대면+사전녹화강의",  #!
    "대면+실시간 화상강의+사전녹화강의",  #!
    "실시간 화상강의+사전녹화강의",    #!
    "100%실시간 화상강의",  #!
    "100%사전녹화강의",  #!
    "실험/실습 실기",
    "외국어 강의", 
    "원격수업",  #!
    "TBL강의",
    "PBL강의",
    "토론식 수업(세미나 포함)프로젝트",
    "실기(예체능)",
    "기타",
    "수강일정",
    "수강일정2",
    "리스트",
    "에러"
]

In [56]:
import os

# 'cp949' 인코딩을 사용하여 CSV 파일을 다시 로드해봅니다.
file_path = r'.\libs\csv\kwu-24summer-timetable_v1.csv'

data_pf = pd.read_csv(file_path, encoding='utf-8')

# 데이터의 상위 몇 개 행을 확인
print(data_pf.head())


   Unnamed: 0  순번  연도      학기              학과/교양영역  구분         과목명  학점   강의시간  \
0           0   1  24  summer             전체공통(정보)  교필      C프로그래밍   3  1,2교시   
1           1   2  24  summer  전체공통\n(글로벌문화와제2외국어)  교선      초급중국어1   3    15일   
2           2   3  24  summer  전체공통\n(글로벌문화와제2외국어)  교선  일본애니메이션과문화   3  3,4교시   
3           3   4  24  summer  전체공통\n(글로벌문화와제2외국어)  교선      초급일본어1   3  3,4교시   
4           4   5  24  summer          전체공통(예술과체육)  교선     대중음악의역사   3    15일   

  담당교수               비 고                                           수강가능한 학과  \
0  정진범               NaN  E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...   
1  곡효운  원격강의 (사전녹화 100%)  E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...   
2  유양근   원격강의 (사전녹화 87%)  E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...   
3  김희성               NaN  E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...   
4  김철수  원격강의 (사전녹화 100%)  E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...   

  전공 인정 학과            테마 이러닝_담당교수     

In [57]:
# Creating an empty DataFrame with the specified columns
df = pd.DataFrame(columns=labels_list)
print(df)

# 빈 행 추가

Empty DataFrame
Columns: [교과목명, 년도학기, 이수구분, 학정번호, 담당교수, 강의소개, 학점/시간, 강의시간 및 강의실, 연락처, 이동전화, 이메일, 담당조교, 장애학생 지원, 교과목 개요, 대표역량, 학습목표 및 학습방법, 학습성과, 영어강의 비율, 출석, 중간고사, 기말고사, 과제보고서, 수업태도, Quiz, 기타1, 영어강의, 중국어 강의, 일본어 강의, TBL강의, PBL강의, 세미나강의, 소규모강의, 융합강의, 팀티칭강의, 일학습병행강의, 외국인전용강의, 서울권역e-러닝, 실험실습실기설계강의, 집중이수제강의, 원격수업 (100%), 원격수업 (50% 이상), 100%대면강의, 대면+실시간 화상강의, 대면+사전녹화강의, 대면+실시간 화상강의+사전녹화강의, 실시간 화상강의+사전녹화강의, 100%실시간 화상강의, 100%사전녹화강의, 실험/실습 실기, 외국어 강의, 원격수업, TBL강의, PBL강의, 토론식 수업(세미나 포함)프로젝트, 실기(예체능), 기타, 수강일정, 수강일정2, 리스트, 에러]
Index: []

[0 rows x 60 columns]


In [58]:
# '담당교수' 열의 'Null' 값을 공란으로 변경
#data_pf['담당교수'] = data_pf['담당교수'].replace('Null', '')

data_pf['담당교수'] = data_pf['담당교수'].fillna('')

i=0
for i in range(len(data_pf)):
    #재실행시 에러가 있는 행만 다시 실행
    #if data_pf['에러']=='':
    #    continue
    df.loc[i] = [np.nan] * len(df.columns)  # 모든 컬럼에 NaN 값으로 빈 행 채우기
    professor_name = data_pf['담당교수'].iloc[i]
    lecture_name = data_pf['과목명'].iloc[i]
    # lecture_code = data_pf['학정번호(lecNumber)'].iloc[i]


    try:
        # 강의계획서 조회 페이지로 이동
        lecture_plan_url = 'https://klas.kw.ac.kr/std/cps/atnlc/LectrePlanStdPage.do'
        driver.get(lecture_plan_url)

        year_select = Select(driver.find_element(By.ID, 'selectYear'))
        year_select.select_by_value(year)

        # 학기 선택
        semester_select = Select(driver.find_element(By.ID, 'selecthakgi'))
        semester_select.select_by_value(semester)


        professor_input = driver.find_element(By.XPATH, '//th[text()="담당교수"]/following-sibling::td/input')
        professor_input.send_keys(professor_name)

        lecture_input = driver.find_element(By.XPATH, '//th[text()="과목명"]/following-sibling::td/input')  
        lecture_input.send_keys(lecture_name)

        # 조회 버튼 클릭
        search_button= driver.find_element(By.XPATH, '//div[@class="btnCon"]/button[@class="btn2 btn-gray"]')
        search_button.click()

        time.sleep(1)

        # 조회 결과 테이블에서 style 속성이 "cursor: pointer;"인 <tr> 요소 내부 데이터 추출
        rows = driver.find_elements(By.XPATH, '//tr[@style="cursor: pointer;"]')
        data = []

        #
        if rows:
            target_row = rows[0]
            target_row.click()  # 선택한 <tr> 요소

        time.sleep(2)
        # 현재 페이지의 내용(텍스트) 가져오기
        page_content = driver.find_element(By.TAG_NAME, 'body').text

        # 주어진 텍스트
        text =page_content.replace(',', '')

        # 정규 표현식을 사용하여 각 항목 추출하는 함수
        def extract_info(pattern, text):
            match = re.search(pattern, text)
            return match.group(1).strip() if match else ""

        # 항목별 정규 표현식 패턴
        patterns = {
            "담당교수": r'담당교수</th>\s<td rowspan="3">(.*?)</td>\s<th rowspan="3">구분',
            "교과목명": r'교과목명\s(.*?)\s년도학기',
            "년도학기": r'년도학기\s(.*?)\s학정번호',
            "이수구분": r'이수구분\s(.*?)\s강의소개',
            "학정번호": r'학정번호\s(.*?)\s이수구분',
            "강의소개": r'강의소개\s(.*?)\s동영상',
            "학점/시간": r'학점/시간\s(.*?)\s강의시간',
            "강의시간 및 강의실": r'\(강의실\)\s(.*?)\s수강인원',
            "연락처": r'연락처\s(.*?)\s이동전화',
            "이동전화": r'이동전화\s(.*?)\s이메일',
            "이메일": r'이메일\s(.*?)\n'
        }

        df.iloc[i, df.columns.get_loc('담당교수')] = extract_info(patterns["담당교수"], text)
        df.iloc[i, df.columns.get_loc('교과목명')] = extract_info(patterns["교과목명"], text)
        df.iloc[i, df.columns.get_loc('년도학기')] = extract_info(patterns["년도학기"], text)
        df.iloc[i, df.columns.get_loc('이수구분')] = extract_info(patterns["이수구분"], text)
        df.iloc[i, df.columns.get_loc('학정번호')] = extract_info(patterns["학정번호"], text)
        df.iloc[i, df.columns.get_loc('강의소개')] = extract_info(patterns["강의소개"], text)
        df.iloc[i, df.columns.get_loc('학점/시간')] = extract_info(patterns["학점/시간"], text)
        df.iloc[i, df.columns.get_loc('강의시간 및 강의실')] = extract_info(patterns["강의시간 및 강의실"], text)
        df.iloc[i, df.columns.get_loc('연락처')] = extract_info(patterns["연락처"], text)
        df.iloc[i, df.columns.get_loc('이동전화')] = extract_info(patterns["이동전화"], text)
        df.iloc[i, df.columns.get_loc('이메일')] = extract_info(patterns["이메일"], text)

        # 정규 표현식을 사용하여 각 항목 추출하는 함수
        def extract_info(pattern, text):
            match = re.search(pattern, text, re.DOTALL)
            return match.group(1).strip() if match else ""

        # 항목별 정규 표현식 패턴
        patterns = {
            "담당조교": r'담당조교\s(.*?)\s',
            "장애학생 지원": r'장애학생 지원\s(.*?)\s교과목',
            "교과목 개요": r'개요\s(.*?)\s교과목\s대표역량',
            "대표역량": r'대표역량\s(.*?)\s학습목표 및',
            "학습목표 및 학습방법": r'학습목표 및\s학습방법\s(.*?)\s교과목',
            "학습성과": r'학습성과\s(.*?)\s강의유형',
        }

        df.iloc[i, df.columns.get_loc('담당조교')] = extract_info(patterns["담당조교"], text)
        df.iloc[i, df.columns.get_loc('장애학생 지원')] = extract_info(patterns["장애학생 지원"], text)
        df.iloc[i, df.columns.get_loc('교과목 개요')] = extract_info(patterns["교과목 개요"], text)
        df.iloc[i, df.columns.get_loc('대표역량')] = extract_info(patterns["대표역량"], text)
        df.iloc[i, df.columns.get_loc('학습목표 및 학습방법')] = extract_info(patterns["학습목표 및 학습방법"], text)
        df.iloc[i, df.columns.get_loc('학습성과')] = extract_info(patterns["학습성과"], text)


        # 정규 표현식을 사용하여 각 항목 추출하는 함수
        def extract_info(pattern, text):
            match = re.search(pattern, text)
            return match.group(1).strip() if match else ""

        # 영어강의 비율 추출
        df.iloc[i, df.columns.get_loc('영어강의 비율')] = extract_info(r'영어강의 비율\s(\d+ %)', text)

        # 평가방법 비율 추출
        evaluation_methods = extract_info(r'평가방법 비율\n출석 중간고사 기말고사 과제보고서 수업태도 Quiz 기타\n(.*?)\n기타평가', text)
        evaluation_values = evaluation_methods.split()

        # 각 평가 항목에 대한 값을 딕셔너리에 저장
        evaluation_dict = {
            "출석": evaluation_values[0],
            "중간고사": evaluation_values[1],
            "기말고사": evaluation_values[2],
            "과제보고서": evaluation_values[3],
            "수업태도": evaluation_values[4],
            "Quiz": evaluation_values[5],
            "기타1": evaluation_values[6]
        }

        for key, value in evaluation_dict.items():
            df.iloc[i, df.columns.get_loc(key)] = value



        # df['영어강의'] = 
        # df['중국어 강의'] = 
        # df['일본어 강의'] = 
        # df['TBL강의'] = 
        # df['PBL강의'] = 
        # df['세미나강의'] = 
        # df['소규모강의'] = 
        # df['융합강의'] = 
        # df['팀티칭강의'] = 
        # df['일학습병행강의'] = 
        # df['외국인전용강의'] = 
        # df['서울권역e-러닝'] = 
        # df['실험실습실기설계강의'] = 
        # df['집중이수제강의'] = 
        # df['원격수업 (100%)'] = 
        # df['원격수업 (50% 이상)'] = 
        # df['100%대면강의'] = 
        # df['대면+실시간 화상강의'] = 
        # df['대면+사전녹화강의'] = 
        # df['대면+실시간 화상강의+사전녹화강의'] = 
        # df['실시간 화상강의+사전녹화강의'] = 
        # df['100%실시간 화상강의'] = 
        # df['100%사전녹화강의'] = 
        # df['실험/실습 실기'] = 
        # df['외국어 강의'] = 
        # df['원격수업'] = 
        # df['TBL강의'] = 
        # df['PBL강의'] = 
        # df['토론식 수업(세미나 포함)프로젝트'] = 
        # df['실기(예체능)'] = 
        # df['기타2수강일정'] = 
        # 결과 출력


        # body 태그의 내용 가져오기
        body_content = driver.find_element(By.TAG_NAME, 'body').get_attribute('innerHTML')


        # body 내에서 input과 label 요소들 찾기
        inputs = driver.find_elements(By.TAG_NAME, 'input')
        labels = driver.find_elements(By.TAG_NAME, 'label')

        # 각 체크박스와 라벨 텍스트 출력
        for input_elem, label in zip(inputs, labels):
            checkbox_state = "checked" if input_elem.is_selected() else "unchecked"
            checkbox_disabled = "disabled" if input_elem.get_attribute("disabled") else "enabled"
            df.iloc[i, df.columns.get_loc(label.text.strip())] = checkbox_state

                
        # 모든 tr 요소 찾기
        all_tr_elements = driver.find_elements(By.TAG_NAME, "table")
        
        if all_tr_elements:
            # 마지막 tr 요소 선택
            last_tr = all_tr_elements[-1]
            df.iloc[i, df.columns.get_loc('수강일정')] = last_tr.text
            last_tr_html = last_tr.get_attribute('outerHTML')
            df.iloc[i, df.columns.get_loc('수강일정2')] = last_tr_html
    except Exception as e:
        df.iloc[i, df.columns.get_loc('에러')] = e


C:\Users\user\AppData\Local\Temp\ipykernel_20440\1817227699.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[i, df.columns.get_loc('담당교수')] = extract_info(patterns["담당교수"], text)
C:\Users\user\AppData\Local\Temp\ipykernel_20440\1817227699.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'C프로그래밍-C Programming' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[i, df.columns.get_loc('교과목명')] = extract_info(patterns["교과목명"], text)
C:\Users\user\AppData\Local\Temp\ipykernel_20440\1817227699.py:80: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024/3' has dtype incompatible with float64, pleas

In [59]:
df.head()

,교과목명,년도학기,이수구분,학정번호,담당교수,강의소개,학점/시간,강의시간 및 강의실,연락처,이동전화,...,원격수업,TBL강의,PBL강의,토론식 수업(세미나 포함)프로젝트,실기(예체능),기타,수강일정,수강일정2,리스트,에러
0,C프로그래밍-C Programming,2024/3,,0000-1-0019-01,,,3/3,월 12교시 (연204) 화 12교시 (연204) 수 12교시 (연204) 목 12...,01056175737,010-5617-5737,...,unchecked,checked,unchecked,unchecked,unchecked,unchecked,평가유형 출석 중간시험 기말시험 과제물 수업태도 Quiz 기타\n평가반영율(%) 2...,"<table class=""AType""><tbody><tr><th style=""wid...",NaN,NaN
1,초급중국어1-Beginning Chinese 1,2024/3,교선,0000-1-2950-01,,,3/3,토 1교시 (미지정),9408366,01086462826,...,checked,unchecked,checked,unchecked,unchecked,unchecked,"주차 강의내용 강의운영방식\n1 중국어에 관한 기본 지식, 중국어 발음\n2 중국어...","<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN
2,일본애니메이션과문화-Japanese animation and Culture,2024/3,교선,0000-1-7363-01,,,3/3,월 34교시 (한울304) 화 34교시 (한울304) 수 34교시 (한울304) 목...,010-6302-0608,010-6302-0608,...,unchecked,unchecked,unchecked,unchecked,unchecked,unchecked,주차 강의내용 강의운영방식\n1 일본 애니메이션의 역사와 현황\n사전 녹화 동영상 ...,"<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN
3,초급일본어1-Elementary Japanese 1,2024/3,교선,0000-1-3593-01,,,3/3,월 34교시 (한울307) 화 34교시 (한울307) 수 34교시 (한울307) 목...,,010-8000-5069,...,unchecked,unchecked,unchecked,unchecked,unchecked,unchecked,주차 강의내용 강의운영방식\n1 강의 개요 \n 제1과 일본어의 문자와 오십음도\n...,"<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN
4,대중음악의역사-History of Pop Music,2024/3,교선,0000-2-6948-01,,,3/3,토 2교시 (미지정),,02-940-8109,...,checked,unchecked,unchecked,checked,unchecked,unchecked,"주차 강의내용 강의운영방식\n1 오리엔테이션 수업 전반에 대한 PREVIEW, 과제...","<table class=""AType""><tbody><tr><th width=""10%...",NaN,NaN


In [60]:
# CSV 파일로 저장   
df.to_csv('./lecture_plan.csv', index=False, encoding='cp949')

에러종류 확인

In [61]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('lecture_plan.csv',encoding='cp949')

# '에러' 칼럼이 비어있지 않은 행들만 필터링
error_df = df[df['에러'].notna()]

# 처음 몇 행 확인
print(error_df.head())

#강의목록에 안뜨면 리스트 오류가 발생하는건가?


   교과목명 년도학기 이수구분 학정번호  담당교수  강의소개 학점/시간 강의시간 및 강의실  연락처 이동전화  ... 원격수업  \
24  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN  NaN  NaN  ...  NaN   
26  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN  NaN  NaN  ...  NaN   
33  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN  NaN  NaN  ...  NaN   
34  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN  NaN  NaN  ...  NaN   
36  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN  NaN  NaN  ...  NaN   

   TBL강의.1 PBL강의.1 토론식 수업(세미나 포함)프로젝트  실기(예체능)   기타 수강일정  수강일정2  리스트  \
24     NaN     NaN                NaN      NaN  NaN  NaN    NaN  NaN   
26     NaN     NaN                NaN      NaN  NaN  NaN    NaN  NaN   
33     NaN     NaN                NaN      NaN  NaN  NaN    NaN  NaN   
34     NaN     NaN                NaN      NaN  NaN  NaN    NaN  NaN   
36     NaN     NaN                NaN      NaN  NaN  NaN    NaN  NaN   

                                                   에러  
24  Alert Text: 폐강된 강의 입니다.\nMessage: unexpected a...  
26  

In [62]:
# '에러' 칼럼의 유니크한 값 확인
unique_errors = error_df['에러'].unique()
print(unique_errors)

['Alert Text: 폐강된 강의 입니다.\nMessage: unexpected alert open: {Alert text : 폐강된 강의 입니다.}\n  (Session info: chrome=127.0.6533.120)\nStacktrace:\n\tGetHandleVerifier [0x00007FF74D919642+30946]\n\t(No symbol) [0x00007FF74D8CE3D9]\n\t(No symbol) [0x00007FF74D7C6FDA]\n\t(No symbol) [0x00007FF74D85B7CC]\n\t(No symbol) [0x00007FF74D83C813]\n\t(No symbol) [0x00007FF74D80A6E5]\n\t(No symbol) [0x00007FF74D80B021]\n\tGetHandleVerifier [0x00007FF74DA4F84D+1301229]\n\tGetHandleVerifier [0x00007FF74DA5BDC7+1351783]\n\tGetHandleVerifier [0x00007FF74DA52A13+1313971]\n\tGetHandleVerifier [0x00007FF74D94DD16+245686]\n\t(No symbol) [0x00007FF74D8D759F]\n\t(No symbol) [0x00007FF74D8D3814]\n\t(No symbol) [0x00007FF74D8D39A2]\n\t(No symbol) [0x00007FF74D8CA3FF]\n\tBaseThreadInitThunk [0x00007FFBD82D257D+29]\n\tRtlUserThreadStart [0x00007FFBD99CAF28+40]\n'
 'Alert Text: 강의 계획서 정보가 없습니다!\nMessage: unexpected alert open: {Alert text : 강의 계획서 정보가 없습니다!}\n  (Session info: chrome=127.0.6533.120)\nStacktrace:\n\tGetH

In [63]:
# 에러 메시지에 따라 레이블을 매핑하는 함수
def categorize_error(error_message):
    # error_message가 NaN이거나 None인 경우, 즉시 '기타' 반환
    if pd.isna(error_message):
        return '기타'
    
    for idx, error in enumerate(unique_errors):
        if error in error_message:
            if idx == 0:
                return '리스트'
            elif idx == 1 or idx == 4:
                return '웹 상호작용 불가능'
            elif idx == 2:
                return '폐강'
            elif idx == 3:
                return '강의 계획서 없음'
    return '기타'  # 에러 메시지가 목록에 없는 경우


# 새로운 칼럼 '에러1'에 레이블 추가
df['에러1'] = df['에러'].apply(categorize_error)


In [64]:
# CSV 파일로 저장   
df.to_csv('./lecture_plan2.csv', index=False, encoding='cp949')

In [65]:
lecture_plan2.head()

NameError: name 'lecture_plan2' is not defined